In [ ]:
import numpy as np
from scipy.special import expit

# A plain vanilla neural-network where each node (perceptron) in a layer is connected to every other node in the subsequent layer
class NeuralNetwork:
  def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
    # Setting the number of nodes in each layer
    self.input_nodes = input_nodes
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes
    # learning_rate controls how far we jump or how much we change weight values so as to not jump around local minima (error) during gradient descent
    self.learning_rate = learning_rate
    # sigmoid or logistic function is used as a threshold to control or supress output signals from each node if they're not large enough { 1/(1+e^-x) }
    # sigmoid outputs are in the range of (0,1) exclusive
    self.activation_function = lambda x: expit(x)

    # Sampling initial weight matrix values from a normal distribution (from bell curve) with mean = 0, standard deviation = 1/math.sqrt(indegree of a node)
    # ih = weights between input and hidden layers, ho = weights between hidden and output layers 
    # Each row in the weight matrix represents a node in the second layer and each column represents every other node in the first layer
    self.weights_ih = np.random.normal(0.0, pow(self.input_nodes, -0.5), (self.hidden_nodes, self.input_nodes))
    self.weights_ho = np.random.normal(0.0, pow(self.hidden_nodes, -0.5), (self.output_nodes, self.hidden_nodes))

  def feed_forward(self, inputs_list):
    # Each column in the inputs_array represent a test sample instance
    inputs_array = np.array(inputs_list, ndmin = 2).T

    # linear combination of weights and input signals = x
    hidden_layer_inputs = np.dot(self.weights_ih, inputs_array)

    # x is fed into sigmoid function (present in the node itself) to check if the input signal is stong enough for a node to fire (squish the values)
    hidden_layer_outputs = self.activation_function(hidden_layer_inputs)

    output_layer_inputs = np.dot(self.weights_ho, hidden_layer_outputs)
    final_outputs = output_layer_outputs = self.activation_function(output_layer_inputs)

    # outputs from each layer is also returned because we need them during back-propagation
    return (inputs_array, hidden_layer_outputs, final_outputs)

  def train_weights(self, inputs_list, targets_list):
  
    inputs_array, hidden_outputs, final_outputs = self.feed_forward(inputs_list)
    targets_array = np.array(targets_list, ndmin = 2).T
    final_output_errors = targets_array - final_outputs

    # (Back-propagation of Errors)
    # Each column in the transposed weight matrix represents every other node in the second layer and row represents a node in the first layer
    # final_output_errors are in the range of (0,1)
    # contribution of the link weights from each node in the first layer is calculated by taking a dot product of (output_errors = proportions) and each row in the transposed weights matrix
    hidden_layer_errors = np.dot(self.weights_ho.T, final_output_errors)
    
    # (Applying Gradient Descent)
    # Update the weights controlled by learning rate and changed by the slope or derivative of the error cost function (y = errors, x = all contributing weights)
    self.weights_ho += self.learning_rate * np.dot((final_output_errors * final_outputs * (1.0 - final_outputs)), hidden_outputs.T)
    self.weights_ih += self.learning_rate * np.dot((hidden_layer_errors * hidden_outputs * (1.0 - hidden_outputs)), inputs_array.T)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.ndimage
%matplotlib inline

def fit_transform(ob, isInput = True):
  if isInput:
    return np.asfarray(ob)/255.0 * 0.99 + 0.01 
  else:
    output = np.zeros(10) + 0.01
    output[ob] = 0.99
    return output

def train(instance, train_input, train_label):
  for idx in range(len(train_inputs)):
    train_input = fit_transform(train_inputs[idx])
    train_label = fit_transform(train_labels[idx], isInput = False)
    instance.train_weights(train_input, train_label)

    plusx_train_input = scipy.ndimage.interpolation.rotate(train_input.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
    instance.train_weights(plusx_train_input.reshape(784), train_label)

    minusx_train_input = scipy.ndimage.interpolation.rotate(train_input.reshape(28,28), -10, cval=0.01, order=1, reshape=False)
    instance.train_weights(minusx_train_input.reshape(784), train_label)

def accuracy(instance, test_file_name):
  test_df = pd.read_csv(test_file_name)
  test_inputs, test_labels = test_df.drop("label", axis = 1).to_numpy(), test_df["label"].to_numpy()
  true_count = 0
  for idx in range(len(test_inputs)):
    (inputs_array, hidden_layer_outputs, final_outputs) = instance.feed_forward(fit_transform(test_inputs[idx]))
    true_count += 1 if np.argmax(final_outputs) == test_labels[idx] else 0
  return true_count / len(test_inputs)

instance = NeuralNetwork(784, 200, 10, 0.2)
train_df = pd.read_csv("mnist_train.csv")
train_inputs, train_labels = train_df.drop("label", axis = 1).to_numpy(), train_df["label"].to_numpy()

epochs = 7
for _ in range(epochs):
  train(instance, train_inputs, train_labels)
print(accuracy(instance, "mnist_test.csv"))




0.9639
